# Imports

In [1]:
from datetime import datetime
import tabula
tabula.environment_info()
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

Python version:
    3.9.18 (main, Nov  2 2023, 17:01:24) 
[Clang 14.0.0 (clang-1400.0.29.202)]
Java version:
    openjdk version "15.0.2" 2021-01-19
OpenJDK Runtime Environment AdoptOpenJDK (build 15.0.2+7)
OpenJDK 64-Bit Server VM AdoptOpenJDK (build 15.0.2+7, mixed mode, sharing)
tabula-py version: 2.8.2
platform: macOS-12.7.4-x86_64-i386-64bit
uname:
    uname_result(system='Darwin', node='Pro.local', release='21.6.0', version='Darwin Kernel Version 21.6.0: Wed Jan 31 21:08:42 PST 2024; root:xnu-8020.240.18.707.2~1/RELEASE_X86_64', machine='x86_64')
linux_distribution: ('Darwin', '21.6.0', '')
mac_ver: ('12.7.4', ('', '', ''), 'x86_64')


# PDF class

In [ ]:
# Transaction class
transaction = Transaction(

# Updated PDFTableExtractor with Transaction output
class PDFTableExtractor:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def extract_table(self, pdf_path):
        pdf_nested_dict = {}
        dfs = tabula.read_pdf(self.file_path, stream=True, pages="all")
        index_correction = 0
        for index, df in enumerate(dfs):
            if df.empty:
                index_correction +=1
            elif not df.empty:
                pdf_nested_dict[index - index_correction] = df
        # print(pdf_nested_dict)      
        return pdf_nested_dict

# Updated BankStatementPDFTableExtractor
class BankStatementPDFTableExtractor(PDFTableExtractor):
    def extract_table(self, pdf_path):
        # Use Tabula to extract table from BankStatement PDF
        # Determine the interpretation (e.g., based on some criteria)
        interpretation = self.determine_interpretation(pdf_path)

        # Extract table with dynamic column names based on interpretation
        if interpretation == "Interpretation1":
            return self.extract_table_interpretation_1(pdf_path)
        elif interpretation == "Interpretation2":
            return self.extract_table_interpretation_2(pdf_path)
        # Add more conditions for other interpretations

    def determine_interpretation(self, pdf_path):
        # Add logic to determine the interpretation based on some criteria
        return interpretation

    def extract_table_interpretation_1(self, pdf_path):
        # Add logic to extract table with column names for Interpretation1
        raw_dataframe = extracted_dataframe_interpretation_1

        # Convert raw DataFrame to a list of Transaction objects
        transactions = self.convert_to_transactions(raw_dataframe)

        return transactions

    def extract_table_interpretation_2(self, pdf_path):
        # Add logic to extract table with column names for Interpretation2
        raw_dataframe = extracted_dataframe_interpretation_2

        # Convert raw DataFrame to a list of Transaction objects
        transactions = self.convert_to_transactions(raw_dataframe)

        return transactions

    def convert_to_transactions(self, raw_dataframe):
        # Implement logic to convert raw DataFrame to a list of Transaction objects
        transactions = []

        for index, row in raw_dataframe.iterrows():
            # Extract required fields
            date = row['Date']
            description = row['Description']
            amount = row['Amount']

            # Extract additional attributes if needed
            additional_attributes = self.extract_additional_attributes(row)

            # Create Transaction object
            transaction = Transaction(date, description, amount, additional_attributes)

            transactions.append(transaction)

        return transactions

    def extract_additional_attributes(self, row):
        # Implement logic to extract additional attributes from the row
        # Return a dictionary of additional attributes
        additional_attributes = {}
        # Add more logic as needed
        return additional_attributes

# Usage
source = "BankStatement"
pdf_path = "path/to/bank_statement.pdf"

factory = PDFTableExtractorFactory()
extractor = factory.create_extractor(source)
transactions = extractor.extract_table(pdf_path)

# Now you have a list of Transaction objects, each with a standardized structure


# WealthSimple Transaction

## Get the transation data

### from disk

In [2]:
# Specify the path to the directory
celi_directory_path = 'data/WealthSimple/CELI/'
cash_directory_path = 'data/WealthSimple/cash/'

path_list = [celi_directory_path, cash_directory_path]
# Get a list of file names in the directory
file_name_dict = {'celi':{'path': 'data/WealthSimple/CELI/',
                          'files':[] }, 
                  'cash':{'path': 'data/WealthSimple/cash/',
                          'files':[] }, 
                 }
# get the path of the files in the dictionary
for k, data in file_name_dict.items():
    for file in os.listdir(data['path']):
        if file[-3:]=="pdf":
            data['files'].append(file)
        
# file_name_dict


### to df

In [6]:
cash_col_name_list = ["CREDIT", "DEBIT", "BALANCE"]

def sort_debit_credit(df):
    credit_list = []
    debit_list = []
    for amount in df["AMOUNT"]:
        # print(amount[0])
        if amount[0] !="$":
            credit_list.append(None)
            debit_list.append(amount)
        else:
            credit_list.append(amount)
            debit_list.append(None)
    df["CREDIT"] = credit_list
    df["DEBIT"] = debit_list
    return df
                    
def clean_amount_col(cash_col_name_list, df):
    caracter_list = ["$", ",", '-']
    for col_name in cash_col_name_list:
        amount_list = []
        for amount in df[col_name]:
            if amount:
                for caracter in caracter_list:
                    amount = amount.replace(caracter, "")
                try:
                    amount = float(amount)
                except:
                    amount = float(amount[1:])
            amount_list.append(amount)
        df[col_name] = amount_list
    return df

def g(s):
    df = pd.DataFrame([x.split() for x in s])
    df.columns = ['date', 'transaction_code', 'description']
    df['description'] = df['description'].apply(lambda x: ' '.join(x[2:]))
    return df

def extract_info_from_Activity_Current_period_col(Activity_Current_period_col, col_name_list):
    data_list = []
    for desc in Activity_Current_period_col:
        date_operation_desc_list = desc.split()
        data_list.append([date_operation_desc_list[0], 
                          date_operation_desc_list[1], 
                          " ".join(date_operation_desc_list[2:])
                         ])
    df_right = pd.DataFrame(data_list, columns=col_name_list)
    return df_right
    
def get_transaction_df(file_name_dict):
    col_name = ["DATE", "DESCRIPTION", "AMOUNT", "BALANCE"]
    num_row = 0
    df_list = []
    for filename in file_name_dict['cash']["files"]:
        file_path = file_name_dict['cash']["path"] + filename
        dfs = tabula.read_pdf(file_path, stream=True, pages="all")
        # print(f"File : {filename} has {len(dfs)} dataframe ")
        
        for i, df in enumerate(dfs):
            # print(f"  df {i} as {len(df.columns)} columns")
            
            if len(df.columns) == 4:
                if df.columns.to_list() == ['Activity - Current period', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']:
                    # format columns
                    # all_col_name_list = [x for x in df.loc[0]]
                    # col_name_list = all_col_name_list[0].split()
                    # other_col_name_list = all_col_name_list[1:]
                    # old_name_col = df.columns.to_list()[1:4]
                    # df.rename(columns={x:y for x,y in zip(old_name_col, other_col_name_list)}, inplace=True)
                    
                    # df_copy = df[1:].copy()
                    # df_copy.reset_index(inplace=True)
                    # df_copy.drop("index", axis=1, inplace=True)
                    # # print(df_copy)
                    
                    # df_right = extract_info_from_Activity_Current_period_col(df_copy['Activity - Current period'], col_name_list)
                    
                    # joined_df = pd.concat([df_copy, df_right], axis=1)
                    
                    # print(joined_df)
                    pass
                else:
                    # pass
                    # print(df.head())
                    col_name = ["DATE", "DESCRIPTION", "AMOUNT", "BALANCE"]
                    values = df.columns.to_list()
                    first = pd.DataFrame({k:[v] for k,v in zip(col_name, values)})
                    df.columns = col_name
                    stacked_df = pd.concat([first, df], axis=0, ignore_index=True)
                    num_row += len(stacked_df)
                    stacked_df["DATE"] = pd.to_datetime(stacked_df["DATE"], format='%Y-%m-%d')
                    stacked_df = sort_debit_credit(stacked_df)
                    stacked_df = clean_amount_col(cash_col_name_list, stacked_df)
                    # print(stacked_df)
                    df_list.append(stacked_df)
        # print("----------------------------------")
    print(num_row)
    df_all = pd.concat(df_list, axis=0)
    df_all.reset_index(inplace=True)
    df_all.drop('index', axis=1, inplace=True)
    
    return df_all

transaction_df = get_transaction_df(file_name_dict)    
# print(transaction_df)


    

310


## Look at the merchants

In [7]:
merchant_list = list(transaction_df["DESCRIPTION"].sort_values().unique())
merchant_list

['Adonis Anjou',
 'Agence De Mobilite Dur',
 'Bikini Village',
 'Brunet Plus Anjou',
 'Bulk Barn',
 'Cashback received',
 'Charcuterie De Tours (',
 'Cinema Triomphe Inc.',
 'Costco Wholesale W525',
 'Couchetard',
 'Deposit',
 'Dollarama',
 'Ferme Sarrazin',
 'Grumpys Bar',
 'Iga',
 'Interest earned',
 'Jean Coutu 045',
 'Jean Coutu 267',
 'Kim Phat',
 'Kumamoto Ramen',
 "L' Echoppe Des Fromage",
 "L'atelier Du Pain",
 'La Vignoble Le Cep Dar',
 'Les Fraiches - Epiceri',
 'Marche Notre Dame 2225',
 'Marche St Jean Baptist',
 'Marche Toit Rouge',
 'Maxi',
 'Maxi & Cie Repentigny',
 "Mcdonald's",
 'Metro De La Rousselier',
 'Metro Fogarty',
 'Proxi Extra Orford',
 'Quebecloisirs Place',
 'Restaurant Le Boucan',
 'Saq23001 Ste-Catherine',
 'Saq23106carrefour De L',
 'Spanel Crepes Et Gourm',
 'Stm Loge Charlevoix N1',
 'Stm Loge Honor Beaugra',
 'Stm Loge Lionel Groulx',
 'Stm Loge Radisson N101',
 'Super C La Plaine 2598',
 'Super C Pte Aux Trembl',
 'Super C Repentigny',
 'Supermarche J

### Is there new merchant?

In [28]:
from investment.models import Merchant, Budget

for budget in Budget.objects.all():
    if budget.unique_provider_keywords:
        print(budget)
        substrings = [x.lower().strip() for x in budget.unique_provider_keywords.split(",") ]
        print(substrings)
# epicerie = Budget.objects.get(spending_item_name='Épicerie')

# keywords_list = epicerie.unique_provider_keywords.split(",") if epicerie.unique_provider_keywords else []

# keywords_list = [x.lower() for x in keywords_list]

# keywords_list
# # existing_merchant_set =  {merchant.name for merchant in Merchant.objects.all()}

# new_merchant_name_to_insert_list = [m for m in merchant_list if m not in existing_merchant_set]

# new_merchant_name_to_insert_list


Épicerie Variable 150.0
['adonis', 'costco', 'super c', 'iga', 'kim phat', 'les fraiches', 'maxi', 'metro', 'foodhero', 'les fraiches', 'adonis', 'provigo', 'marche richelieu', 'miel morand', 'ferme la vraie vie', 'pasta and fumo', 'flashfood', 'capulus cafe mobile', 'monsieur cocktail', 'nespresso canada']
Transport (electricité / parking / metro-bus) Variable 300.0
['agence de mobilite durablmontreal', 'stm', 'le circuit electrique montreal', 'flo services quebec', 'esso', 'shell', 'petro canada', 'agence de mobilite dur', 'stm']
vin Variable 120.0
['norman hardie winery', 'saq', 'karlo estates', 'lcbo', 'parsons brewing', 'stock and row', 'hinterland wine', 'county cider picton', 'slake brewing picton', 'domaine darius hillier on', 'closson chase vineyard hillier on', 'redtail vineyard', 'rosehall run vineyards', "aux p'tits bocaux"]


### Association mot clef -> catégorie

In [ ]:
# les budget items ont des substring qui permette d'associer des marchand

categories = {
    "Épicerie": ["Adonis", "Costco", "Super C", "Iga", "Kim Phat", "Les Fraiches", "Maxi", "Metro"],
    "Vêtements": ["Bikini Village", "Dollarama", "Suzy Shier"],
    "Restaurants": ["Grumpys Bar", "Kumamoto Ramen", "Mcdonald's", "Restaurant Le Boucan"],
    "Bars": ["Grumpys Bar"],
    "Pharmacies": ["Brunet Plus Anjou", "Jean Coutu 045", "Jean Coutu 267"],
    "Vin": ["Saq23001 Ste-Catherine", "Saq23106carrefour De L", "La Vignoble Le Cep Dar" ],
    "Autres": ["Bulk Barn", "Cashback received", "Charcuterie De Tours", "Cinema Triomphe Inc.", "Deposit", "Ferme Sarrazin", "Interest earned", "L' Echoppe Des Fromage", "L'atelier Du Pain", "Marche Notre Dame 2225", "Marche St Jean Baptist", "Marche Toit Rouge", "Maxi & Cie Repentigny", "Proxi Extra Orford", "Quebecloisirs Place", "Spanel Crepes Et Gourm", "Supermarche J P V Plou", "Tradition", "Walmart"],
    "Transport Collectif": ["Agence De Mobilite Dur", "Stm"]  # Corrected key name and removed extra parenthesis
}


In [29]:
# def categorize_strings(dictionary, string_list):
#     result = {}
#     for string in string_list:
#         category_found = None
#         for category, substrings in dictionary.items():
#             if any(sub in string for sub in substrings):
#                 category_found = category
#                 break
#         result[string] = category_found
#     return result

# result = categorize_strings(categories, new_merchant_name_to_insert_list)

def categorize_strings(Budget, string_list):
    result = {}
    string_list = [x.lower() for x in string_list]
    for string in string_list:
        category_found = None
        for budget in Budget.objects.all():
            if budget.unique_provider_keywords:
                substrings = [x.lower().strip() for x in budget.unique_provider_keywords.split(",") ]
                if any(sub in string for sub in substrings):
                    category_found = budget
                    break
        result[string] = category_found
    return result



result = categorize_strings(Budget, merchant_list)
print(result)

# categorized_merchant = categorize_strings(categories, new_merchant_name_to_insert_list)
# categorized_merchant

{'adonis anjou': <Budget: Épicerie Variable 150.0>, 'agence de mobilite dur': <Budget: Transport (electricité / parking / metro-bus) Variable 300.0>, 'bikini village': None, 'brunet plus anjou': None, 'bulk barn': None, 'cashback received': None, 'charcuterie de tours (': None, 'cinema triomphe inc.': None, 'costco wholesale w525': <Budget: Épicerie Variable 150.0>, 'couchetard': None, 'deposit': None, 'dollarama': None, 'ferme sarrazin': None, 'grumpys bar': None, 'iga': <Budget: Épicerie Variable 150.0>, 'interest earned': None, 'jean coutu 045': None, 'jean coutu 267': None, 'kim phat': <Budget: Épicerie Variable 150.0>, 'kumamoto ramen': None, "l' echoppe des fromage": None, "l'atelier du pain": None, 'la vignoble le cep dar': None, 'les fraiches - epiceri': <Budget: Épicerie Variable 150.0>, 'marche notre dame 2225': None, 'marche st jean baptist': None, 'marche toit rouge': None, 'maxi': <Budget: Épicerie Variable 150.0>, 'maxi & cie repentigny': <Budget: Épicerie Variable 150.0>

In [ ]:
desc_count = dfs_staked["DESCRIPTION"].value_counts()
desc_count

In [ ]:
result = dfs_staked.groupby("DESCRIPTION").agg({
    "DESCRIPTION": 'count',  # Frequency count
    'DEBIT': 'sum',    # Sum of 'col2' values
    'CREDIT': 'sum'     # Sum of 'col3' values (assuming numerical)
})
result = result.rename(columns={'DESCRIPTION': 'Description'})
# .sort_values("DESCRIPTION", ascending=False)
result.sort_values("Description", ascending=False)

In [ ]:
# x = directory_path + data_list[0]
for file_name, d in data_dict.items():
    file_path = directory_path + file_name
    if os.path.isfile(file_path):
        print("is file")
        dfs = tabula.read_pdf(file_path, stream=True, pages="all")
        for index, df in enumerate(dfs):
            print("*********", index)
            col = df.iloc[0].to_list()
            df.columns = col
            df = df[1:]
            data_dict[file_name][index] = df
            
        

## WealthSimple Investment

In [ ]:
from math import isnan
dict_0 = {}
dict_1 = {}
dict_2 = {}
col_1 = ['nan', 'nan', 'Market % of Market', 'Book', '% of Total Book']
col_2 = ['nan', 'Symbol', 'Total Quantity', 'Segregated', 'Market', 'Market', 'Book']
# list_of_col = 
market_value_list = []
percentage_market_value_list = []
for file_name, df_dict in data_dict.items():
    print("____________", file_name)
    print(f"there are {len(dfs)} Dataframes")
    for index, dfe in df_dict.items():
        df = dfe.copy()
        df_col = dfe.columns.to_list()
        if 'Market % of Market' in df_col:
            df = df.iloc[:,1: ]
            df = df[1:]
            df.dropna(inplace=True)
            df.reset_index(inplace=True)
            df.drop("index", axis=1, inplace=True)
            mv_pmv = df["Market % of Market"].str.split()
            # print(mv_pmv.to_list())
            market_value_list = [pd.to_numeric(x[0].replace("$", "").replace(",", "")) for x in mv_pmv.to_list()]
            percentage_market_value_list = [pd.to_numeric(x[1])/100 for x in mv_pmv.to_list()]
            df["Market Value"] = market_value_list
            df["% of Market Value"] = percentage_market_value_list
            df.drop("Market % of Market", axis=1, inplace=True)
            # df["Book"] = pd.to_numeric(df["Book"].replace("$", "").replace(",", "")) 
            df["Book"] = df["Book"].str.replace("$", "", regex=True)
            df["Book"] = pd.to_numeric(df["Book"].str.replace(",", "", regex=True))
            df.rename(columns={"Book": "Book Value", np.nan: "Description", "% of Total Book": "% of Total Book Value"}, inplace=True)
            df["% of Total Book Value"] = pd.to_numeric(df["% of Total Book Value"]) / 100
        elif 'Segregated' in df_col:
            df.rename(columns={np.nan: df.iloc[1, 0], 
                   "Segregated": "Segregated Quantity",
                  }, inplace=True)
            df.dropna(inplace=True)
            df.reset_index(inplace=True)
            df.drop('index', axis=1, inplace=True)
            df.columns = ["Description", "Symbol", "Total Quantity", "Segregated Quantity", "Market Value", "Total Market Value", "Book Value"]
            for col in ["Market Value", "Total Market Value", "Book Value"]:
                df[col] = df[col].str.replace(",", "", regex=True)
                df[col] = df[col].str.replace("$", "", regex=True)
                df[col] = df[col].str.replace("CAD", "", regex=True)
                df[col] = pd.to_numeric(df[col])
        elif 'Last Statement Cash Balance' in df_col:
            # pass/
#              les titres des colone ne sont que la répétitio de la ligne d'avant
            # print(df.columns.to_list())
            for k, v in df.iterrows():
                index_list = v.index.to_list()
                index_list.insert(3, "___")
                row_list = v.to_list()
                # print("index", len(index_list))
                filtered_row = [item for item in row_list if str(item).lower() != 'nan' and (not isinstance(item, float) or not np.isnan(item))]
                # print("row", len(row_list), filtered_row)
        elif 'Index ETF' in df_col:
            # pass
            col_names = ["Description", "Symbol", "Total Quantity", "Segregated Quantity", "Market Value", "Total Market Value", "Book Value"]
            df.columns = col_names
            for col in ["Market Value", "Total Market Value", "Book Value"]:
                df[col] = df[col].str.replace(",", "", regex=True)
                df[col] = df[col].str.replace("$", "", regex=True)
                df[col] = df[col].str.replace("CAD", "", regex=True)
            print(df.dropna())
        elif 'Charged ($)' in df_col:
            pass
        elif 'Segregated Quantity' in df_col:
            pass
        else:
            print(df.columns.to_list())